## 하이퍼 파라미터 튜닝

### 하이퍼 파라미터 튜닝 크게나누면 4가지
1. Grid Search : params를 주고 그리드 모양으로 모두 서치
2. Random Search : 그리드 서치를 랜덤하게 한다.
3. Bayesian Optimization
4. 수동 튜닝

### 하이퍼 파라미터 튜닝의 주요 이슈

<img src=https://blog.kakaocdn.net/dn/cJ0vQo/btrCjFlBrse/Siw2SOMxqgGbuzKQPRAd31/img.png width=1200>

- Gradient Boosting 기반 알고리즘은 튜닝 해야 할 하이퍼 파라미터 개수가 많고, 번위가 넓어서 가능한 개별 경우의 수가 너무 많음.
- 이러한 경우의 수가 많은 경우 데이터가 크면 하이퍼 파라미터 튜닝 수행이 너무 오래 걸림.

-> 주어진 경우의수만 다해보기보단. 성능이 좋아지는 방향으로 최소의 시도를 통해, 좋은 성능의 모델을 뽑아내는것 -> 베이지안 최적화

## 베이지안 최적화

<img src=https://blog.kakaocdn.net/dn/b9Fvq7/btrCkf06S1J/feMCpi5DhoXxwMsYuuRff1/img.png width=1200>

### 베이지안 최적화 수행 단계 4스텝

<img src=https://blog.kakaocdn.net/dn/cpSpoB/btrCkgyS1hR/CgOS8GJEMCvkFk2u9Buej1/img.png width1200>

### 베이지안 최적화를 구현한 주요 패키지
1. HyperOpt: 
2. Bayesian optimization: 
3. Optuna: 

### HyperOpt를 통한 최적화 예시

<img src=https://blog.kakaocdn.net/dn/mjYYx/btrCebrvgeD/7wRIsVYkCpFMcxHX4TgxB1/img.png width=1200>

**구성 요소 3가지**
1. Search Space : 입력값 범위. x와 y의 입력값 범위를 준다.
2. 목적 함수 : 블랙박스 함수. 내부가 어떻게 돌아가는지 모르고 입력값에 대한 반환값을 알 수 있는 함수
3. 목적 함수 반환 최소값 유추 예시 -> **fmin**함수가 핵심
-> objective_func함수에 search_space 범위의 입력값을 넣었을 때, max_evlas번 반복하며 베이지안 최적화로 최소값 유추하는 예시.

    여기선 5번 반복하며 최소값 64.080 찾음

<img src=https://blog.kakaocdn.net/dn/bykjOu/btrCiQOCfIV/flkFRV5K2U8XFNLSn6aGO1/img.png width=1200>

### 베이지안 최적화 개요와 HyperOpt 사용법

In [1]:
import hyperopt
print(hyperopt.__version__)

0.2.7


In [2]:
# !pip install hyperopt==0.2.7

In [45]:
from hyperopt import hp
print(type(hp.quniform('x',-10, 10, 1)))

# x는 -10 ~ 10 까지 1간격을 가지고, y는 -15 ~ 15까지 1간격을 가지는 입력 변수.
# x : -10, -9, ... 9, 10    /    y: -15, -14, ... 14, 15
search_space = {'x': hp.quniform('x', -10, 10, 1), 'y': hp.quniform('y', -15, 15, 1) }
search_space

<class 'hyperopt.pyll.base.Apply'>


{'x': <hyperopt.pyll.base.Apply at 0x12a756640>,
 'y': <hyperopt.pyll.base.Apply at 0x12a756460>}

In [47]:
from hyperopt import STATUS_OK
STATUS_OK

'ok'

In [50]:
# 목적 함수 생성
# : 입력 변수 검색 범위를 갖는 딕셔너리 search_space(x,y)를 받고, x^2 - 20y 값을 반환
def objective_func(search_space):
    x = search_space['x'] # <hyperopt.pyll.base.Apply at 0x12a756640>
    y = search_space['y'] # <hyperopt.pyll.base.Apply at 0x12a756460>
    retval = x**2 - 20*y
    return retval # return {'loss':retval, 'status':STATUS_OK} 해도 된다.

#### HyperOpt에서 베이지안최적화를 진행하는 함수는 fmin

In [55]:
from hyperopt import fmin, tpe, Trials
import numpy as np

# 최적화 과정의 결과값을 저장한 Tirals 객체 생성
trial_val = Trials()

# fmin을 활용해 목적 함수의 최소값 찾기
# 인자
## fn: 목적함수, space: 탐색영역
## algo: search algorithm. 여기서 tpe.suggest를 사용 (Tree Parzen Estimator: SMBO(순차 모델 기반 최적화) 접근 방식)
## max_evals: 반복 횟수
## trials=trial_val : max_evals만큼 수행한 입력값과 결과값을 trail_val에 저장
## rstate : random_state. 여기선 정수 말고, 아래처럼 np.random.default_rng(seed=정수) 형태를 사용.
## => 경험적으로는 rstate를 따로 설정하지 않는게 성능이 더 좋은 경향.
best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5,
              trials=trial_val, rstate=np.random.default_rng(seed=0))
print(f"best: {best_01}")

100%|██████████| 5/5 [00:00<00:00, 556.42trial/s, best loss: -224.0]
best: {'x': -4.0, 'y': 12.0}


In [56]:
trial_val = Trials()

# max_evals를 20회로 늘려서 재테스트 -> 성능이 더 좋아진다.
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20
               , trials=trial_val, rstate=np.random.default_rng(seed=0))
print('best:', best_02)

100%|██████████| 20/20 [00:00<00:00, 941.08trial/s, best loss: -296.0]
best: {'x': 2.0, 'y': 15.0}


In [57]:
trial_val

#### Trials(): HyperOpt 수행, 입력 값들과 목적 함수 반환값 보기

In [61]:
# fmin( )에 인자로 들어가는 Trials 객체의 results 속성에 파이썬 리스트로 목적 함수 반환값들이 저장됨.
# 리스트(results) 내부의 개별 원소는 {'loss':함수 반환값, 'status':반환 상태값}와 같은 딕셔너리.
print(trial_val.results[:3])

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}]


In [73]:
# Trials 객체의 vals 속성에 {'입력변수명': 개별 수행시 입력된 값들의 리스트} 형태로 저장.
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


In [74]:
import pandas as pd

In [80]:
# results에서 loss key값에 해당하는 value들을 추출하여 리스트로
losses = [d['loss'] for d in trial_val.results]
print(losses[:5])

[-64.0, -184.0, 56.0, -224.0, 61.0]


#### Trials() 결과 데이터프레임으로 보기좋게

In [99]:
# DataFrame으로 생성
## trial_val.vals 딕셔너리와 {'losses':losses} 딕셔너리 합치기
sum_dict = dict(trial_val.vals,**{'losses':losses})
result_df = pd.DataFrame(sum_dict)

# DataFrame으로 생성 (강의 버전)
# result_df = pd.DataFrame({'x': trial_val.vals['x'],'y': trial_val.vals['y'],'losses': losses})
result_df.sort_values('losses').head(5)

,x,y,losses
5,2.0,15.0,-296.0
3,-4.0,12.0,-224.0
1,-4.0,10.0,-184.0
18,7.0,11.0,-171.0
13,6.0,10.0,-164.0


### HyperOpt를 XGBoost 하이퍼 파라미터 튜닝에 적용

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target']= dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]

# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label,
                                         test_size=0.2, random_state=156 )

# 학습 데이터를 다시 학습과 검증 데이터로 분리 
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train,
                                         test_size=0.1, random_state=156 )

![](./hyperopt.png)

In [ ]:
from hyperopt import hp

# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 검색. 
# hp.uniform: 정의된 범위 내에서 (정규분포로)임의의 숫자를 무작위 추출
# hp.quniform: 정의된 범위 내에서 마지막 숫자만큼의 간격을 두어 숫자를 추출 
xgb_search_space = {'max_depth
': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2), # learning_rate는 정수형이 아닌, 정규분포 형태
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# fmin()에서 입력된 search_space값으로 입력된 모든 값은 실수형임. 
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함. 
# 정확도는 높은 수록 더 좋은 수치임. -1* 정확도를 곱해서 큰 정확도 값일 수록 최소가 되도록 변환
def objective_func(search_space):
    # 수행 시간 절약을 위해 n_estimators는 100으로 축소
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
        
    # accuracy는 cv=3 개수만큼의 정확도 결과를 가지므로 이를 평균해서 반환하되 -1을 곱해줌. 
    # acc는 높은게 더 좋은건데, fmin은 최소값을 찾으므로 -1 곱하기
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

In [ ]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

In [ ]:
print('colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
                        round(best['colsample_bytree'], 5), round(best['learning_rate'], 5),
                        int(best['max_depth']), int(best['min_child_weight'])))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )

evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='logloss', 
                eval_set=evals, verbose=True)

preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)

In [ ]:
losses = [loss_dict['loss'] for loss_dict in trial_val.results]
result_df = pd.DataFrame({'max_depth': trial_val.vals['max_depth'],
                          'min_child_weight': trial_val.vals['min_child_weight'],
                          'colsample_bytree': trial_val.vals['colsample_bytree'],
                          'learning_rate': trial_val.vals['learning_rate'],
                          'losses': losses
                         }
                        )
result_df